In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('qqp.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('qqp.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.is_duplicate[df1.is_duplicate == 1]="True"
df1.is_duplicate[df1.is_duplicate == 0]="False"

df2.is_duplicate[df2.is_duplicate == 1]="True"
df2.is_duplicate[df2.is_duplicate == 0]="False"

C:\Users\moshi\AppData\Local\Temp/ipykernel_20032/3280091234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.is_duplicate[df1.is_duplicate == 1]="True"
C:\Users\moshi\AppData\Local\Temp/ipykernel_20032/3280091234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.is_duplicate[df1.is_duplicate == 0]="False"
C:\Users\moshi\AppData\Local\Temp/ipykernel_20032/3280091234.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.i

In [4]:
# taking acceptable and deleting unacceptable
df3 = df1[~df1.is_duplicate.str.contains('True')]

In [5]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [6]:
df3["is_duplicate"].value_counts()

False    250
Name: is_duplicate, dtype: int64

In [7]:
# taking unacceptable and deleting unacceptable
df4 = df2[~df2.is_duplicate.str.contains("False")]

In [8]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [9]:
df4["is_duplicate"].value_counts()

True    250
Name: is_duplicate, dtype: int64

In [10]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [11]:
temp["is_duplicate"].value_counts()

False    250
True     250
Name: is_duplicate, dtype: int64

In [12]:
temp.head()

,id,qid1,qid2,question1,question2,is_duplicate
268255,152883,240108,240109,Why do we have to pretend that we are happy?,Is it healthy to pretend to be happy?,False
358369,347538,331600,66631,What is the corporate culture like at Excel Tr...,What is the corporate culture like at Northern...,False
107217,322172,447952,447953,What is the best girls hostel in Kota near sat...,Which are the best hostels in Kota near the An...,False
156862,174712,269183,269184,What if Donald Trump had stood against Barack ...,I am planning to start an online food market m...,False
274590,157654,246447,246448,Why do some people write with their right hand...,Why do people become left-handed or right-handed?,False


In [13]:
temp.tail()

,id,qid1,qid2,question1,question2,is_duplicate
246229,167726,259971,107016,How do you train a Dogue de Bordeaux?,How can I train my dog?,True
57705,197153,7251,65964,Is it possible to get pregnant if a couple is ...,Can you get pregnant a couple days before your...,True
36063,330075,456799,456800,What can I become efficient at math problem so...,What can I do to become efficient at solving m...,True
139731,389844,174340,158723,Why are oceans blue?,Why are the oceans blue?,True
83140,134543,215015,215016,"Now that Trump won, is he gonna build the wall...","Now that he's elected president, will Trump ac...",True


In [16]:
text1 = temp['question1']+temp['question2']
emotion= temp['is_duplicate'].tolist()

In [17]:
text=text1.values.tolist()

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [19]:
import ktrain
from ktrain import text
MODEL_NAME = 'prajjwal1/bert-tiny'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [20]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 36
	99percentile : 51


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 37
	99percentile : 46


In [21]:
model = t.get_classifier()

C:\Users\moshi\anaconda3\lib\site-packages\ktrain\text\preprocessor.py:1069: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


In [22]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [23]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 28s 330ms/step - loss: 0.7021 - accuracy: 0.4875 - val_loss: 0.7015 - val_accuracy: 0.4700
Epoch 2/5
67/67 [==============================] - 21s 318ms/step - loss: 0.6917 - accuracy: 0.5400 - val_loss: 0.6981 - val_accuracy: 0.4800
Epoch 3/5
67/67 [==============================] - 19s 280ms/step - loss: 0.6872 - accuracy: 0.5550 - val_loss: 0.6946 - val_accuracy: 0.5400
Epoch 4/5
67/67 [==============================] - 19s 284ms/step - loss: 0.6786 - accuracy: 0.5950 - val_loss: 0.6971 - val_accuracy: 0.4700
Epoch 5/5
67/67 [==============================] - 24s 353ms/step - loss: 0.6705 - accuracy: 0.6575 - val_loss: 0.6950 - val_accuracy: 0.4900


In [24]:
class_names=["acpt","unac"]

In [25]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        acpt       0.46      0.59      0.51        46
        unac       0.54      0.41      0.46        54

    accuracy                           0.49       100
   macro avg       0.50      0.50      0.49       100
weighted avg       0.50      0.49      0.49       100



array([[27, 19],
       [32, 22]], dtype=int64)